In [ ]:
%%capture
!pip install openmined_psi

In [ ]:
import syft as sy
duet = sy.launch_duet(loopback=True)

In [ ]:
import openmined_psi as psi

In [ ]:
class PsiServerDuet:
    def __init__(self, duet, server_items, reveal_intersection=False, fpr=1e-6):
        # save a reference to the current duet session
        self.duet = duet
        
        # send the reveal intersection flag to the client
        self.duet.requests.add_handler(
            name="reveal_intersection",
            action="accept"
        )
        sy_reveal_intersection = sy.lib.python.Bool(reveal_intersection)
        sy_reveal_intersection_ptr = sy_reveal_intersection.tag("reveal_intersection").send(self.duet, pointable=True)
        
        # allow the client to access the false positive rate
        self.duet.requests.add_handler(
            name="fpr",
            action="accept"
        )
        sy_fpr = sy.lib.python.Float(fpr)
        sy_fpr_ptr = sy_fpr.tag("fpr").send(self.duet, pointable=True)
    
        # start the server
        self.server = psi.server.CreateWithNewKey(reveal_intersection)
        
        # send ServerSetup
        self.duet.requests.add_handler(
            name="setup",
            action="accept"
        )
        setup = self.server.CreateSetupMessage(fpr, 1, server_items)
        setup_ptr = setup.tag("setup").send(self.duet, pointable=True)
    
    def accept(self, timeout_secs=-1):
        # block until a request is received from the client
        while True:
            try:
                self.duet.store["request"]
            except:
                continue
            
            break
        
        # get the Request from the client
        request_ptr = self.duet.store["request"]
        request = request_ptr.get(
            request_block=True,
            name="request",
            reason="To get the client request",
            timeout_secs=timeout_secs,
            delete_obj=True
        )
        
        # process the request and send Response to client
        self.duet.requests.add_handler(
            name="response",
            action="accept"
        )
        response = self.server.ProcessRequest(request)
        response_ptr = response.tag("response").send(self.duet, pointable=True)

In [ ]:
server_items = ["Element " + str(2 * i) for i in range(1000)]

In [ ]:
server = PsiServerDuet(duet, server_items)
# data owner has full control over the number of intersections that can be done
server.accept()